In [ ]:
'''
import your life
'''
import numpy as np
import io
import datetime as date
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
import plotly.graph_objects as go

In [ ]:
def myround(x, base=5):
    """
    Rounds to the nearest 5 by default or any other number.
    """
    return base * round(x/base)

In [ ]:
def sortbyphaz(df,  **kwargs):
    """
    This function sorts the kinetic signals data stream at intersections with detectors. Input kinetic signals device event data as a dataframe.
    There is a second input which should be entered as a **kwargs; this input is the name of the detector and phase, i.e. '3a' for phase 3 and detector a. The function goes up to c for phase inputs and has an extra one for even phases with an ending r.
    The function is also set up to read the device event data in order to show what phase is being served.
    The output of the function is a edited df with columns: ('Event ID', 'Event Value', 'Timestamp', 'Time Between Rows') for only the events implied in the kwargs and the time line.
    """
    #Initialize variables and all of the different kwargs that the phase can be (needs to be a different one every time to calculate the time between detector on/off)
    phases = {'1a': [], '1b': [],'1c':[], '2a': [], '2b': [],'2c':[],'3a':[],'3b':[],'3c':[], '4a': [], '4b': [],'4c':[],
              '5a': [], '5b': [], '5c': [], '6a': [], '6b': [],'6c':[], '7a': [], '7b': [],'7c':[],
              '8a': [], '8b': [],'8c':[],'2r':[],'4r':[],'6r':[],'8r':[]}
    timeline = [] #the timeline are events that are happening in the data stream

    for _, row in df.iterrows():
        event_id = row['Event ID']
        event_value = row['Event Value']

        if 81 <= event_id <= 100: #the values for defectors
            added = False
            for phase, values in kwargs.items():
                if event_value in values: #if the phase matchs the told phase number
                    phases[phase].append(row.copy())  # Append a copy of the row to the list of phases in a dictionary
                    added = True
                    break  # Exit loop if added to a phase
            if not added:
                print(f"Warning: Event Value {event_value} did not match any phase.")

        elif 0 <= event_id <= 20:
            timeline.append(row)

    for key, rows in phases.items():  # Iterate over the phases dictionary
        if rows:  # Check if there are any rows in this phase
            phases[key] = pd.DataFrame(rows)  # Convert the list of rows into a DataFrame
            phases[key] = phases[key].sort_values(by='Timestamp')
            phases[key]['Time Between Rows'] = phases[key]['Timestamp'].diff().dt.total_seconds() #the on/off calculation
        else:
            phases[key] = pd.DataFrame(columns=['Event ID', 'Event Value', 'Timestamp', 'Time Between Rows'])

    timeline_df = pd.DataFrame(timeline)
    timeline_df = timeline_df.sort_values(by='Timestamp')

    # Construct new_df by combining phases and timeline_df
    new_df = pd.concat(list(phases.values()) + [timeline_df])
    new_df = new_df.sort_values(by='Timestamp')

    # Round to the nearest 15 minutes
    new_df['Minuterowd'] = new_df['Timestamp'].apply(
        lambda dt: dt + pd.Timedelta(minutes=myround(dt.minute,15) - dt.minute,seconds=-dt.second, microseconds=-dt.microsecond)
    )

    return new_df

In [ ]:
def traffic_count(rawdata, **kwargs):
    """
    The point of this function is to take the output of the sortbyphaz dataframe and
    the kwargs which is used to indicate which detection device goes to which phase. It has all the numbered phases including phases 2r, 4r, 6r, and 8r to account for right turning movements.
    The output is the phase average for every fifteen minutes and the intersection as a whole, counting in hour long segments but printing in fifteen minute segments for smoother results.
    The function could be modified to account for different detector types, but at the moment it is only counting the off moments of the detector and summing that to the phase.
    """
    # Initialize variables for counting and tracking
    counts = {
        'intersection': 0,
        '1': 0,
        '2': 0,
        '2r': 0,
        '3': 0,
        '4': 0,
        '4r': 0,
        '5': 0,
        '6': 0,
        '6r': 0,
        '7': 0,
        '8': 0,
        '8r': 0
    }

    old = rawdata.iloc[0]['Minuterowd'].minute # Initialize the time variable for the intersection as a whole with the first minute.
    phase_old = {key: rawdata.iloc[0]['Minuterowd'].minute for key in counts.keys() if key != '15'}

    newdf = []

    for c, row in rawdata.iterrows():
        # Traffic counting
      if row['Event Name'] == "Detector Off":
            # Intersection-wide counting
        if row['Minuterowd'] == old: #counting in an non fifteen-minute chunk
          counts['intersection'] += 1
          old= row['Minuterowd']
        else:
          new_row = {
          "Fifteen Minute Average": (counts['intersection'] *4),
          "Start Minute": pd.to_datetime(old),
          "Phase": "intersection"}
          newdf.append(new_row)
          old= row['Minuterowd']
          counts['intersection'] = 1

            # Phases
        for key, value in kwargs.items():
          if row['Event Value'] in value:
            if row['Minuterowd'] == phase_old[key]:
              counts[key] += 1
              phase_old[key] = row['Minuterowd']
            else:
              new_row = {
                        "Fifteen Minute Average": counts[key] * 4,
                        "Start Minute": pd.to_datetime(phase_old[key]),
                        "Phase": key
                        }
              newdf.append(new_row)
              counts[key] = 1
              phase_old[key] = row['Minuterowd']

    box_df = pd.DataFrame(newdf)
    return box_df

In [ ]:
join1=pd.read_csv("DeviceEvents_06-0946_2024-07-23_13-23-21.csv") #read in the data
join2=pd.read_csv("DeviceEvents_06-0946_2024-07-23_13-24-02.csv")
df = pd.concat([join1, join2], ignore_index=True)

In [ ]:
df = df.sort_values(by='Timestamp') #sort the data
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format="%m/%d/%Y %H:%M:%S.%f") #convert the data to datetime
df1=sortbyphaz(df, **{'1a':[1],'1b':[29],'2a':[2],'2b':[3],'4a':[8],'5a':[15],'5b':[31],'5c':[16],'6a':[18],'6b':[19],'8a':[22],'8b':[23]}) #sort the data by phase
df1

In [ ]:
df2=traffic_count(df1,**{'1': [1,29], '2': [2,3], '4': [8], '5': [15,31], '6': [18,19], '8': [22,23], '4r': [16]})#count the phases
df2 = df2.drop(df2[df2['Start Minute'] == '1970-01-01 00:00:00'].index) #getting rid of the initial row
df2['date'] = df2['Start Minute'].apply(lambda x: x.date()) #add date and time columns separate
df2['time'] = df2['Start Minute'].apply(lambda x: x.time())
df2

In [ ]:
#graphing the data for phases
import plotly.express as px
fig = px.line(df2, x='Start Minute', y='Fifteen Minute Average', color='Phase', color_discrete_map={'1': 'red', '2': 'blue', '3':'Green'}) #graff the data
fig.update_layout(title='Traffic Count by Phase', xaxis_title='Date and Time', yaxis_title='Fifteen Minute Average')
fig.show()
df2

In [ ]:
#graphing the data for intersection wide
df3 = df2.sort_values(by='Start Minute', ascending=False) #rotate the graph to go in descending order
df4 = df3[df3['Phase'] == 'intersection'] #filtering for just interesction wide
contour_plot = go.Heatmap(x=df4['date'], y=df4['time'], z=df4["Fifteen Minute Average"],colorscale='Sunset', reversescale=False,
    colorbar=dict(
        title="Traffic Count",
        titleside="top"
    ))
fig = go.Figure(data=[contour_plot])
fig.update_layout(title='Whole Intersection Traffic Count in 15 Minute Increments',xaxis_title='Date', yaxis_title='Time',
    height=650)
fig.show()
#fig.write_html("06-0955heatmap.html") #printing the graph
#like if reverse Blackbody, Spectral
#Sunset, HSV ,mygbm, rdylgn, picnic, Portland
#vary destgt bad colers mrybm